In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import normaltest
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA

import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor

from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold

from catboost import CatBoostRegressor, Pool

import lightgbm as lgb

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [6]:
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.3 MB/s eta 0:00:0000:0100:010m


In [10]:
train = pd.read_csv('train.csv')
test =pd.read_csv('test.csv')

X = train.drop(columns = ['y','id'])
y = train['y']

test_ids = test['id']
test.drop(columns = 'id',inplace = True)

X = X.astype(float)
y = y.astype(float)
test = test.astype(float)

y = np.log(y)

counts = pd.DataFrame(X.nunique()).reset_index()
todrop = counts[counts[0] == 1]['index']

X.drop(columns = todrop,inplace = True)
test.drop(columns = todrop, inplace = True)


cols = X.columns


In [15]:
corrs.iloc[0,0]

1.0

In [11]:
corrs = pd.DataFrame(np.triu(X.corr()))
corrs.columns = cols
corrs.head()

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x756,x757,x758,x759,x760,x761,x762,x763,x764,x765
0,1.0,-0.102542,0.153749,0.379679,0.493197,0.247154,-0.165646,-0.103770,-0.032586,-0.108841,...,-0.010912,-0.052222,-0.006532,-0.016382,-0.057818,0.602138,-0.058439,0.629004,-0.149572,-0.005274
1,0.0,1.000000,-0.177286,0.035330,-0.180123,0.071401,0.463602,0.961286,0.103345,0.257011,...,-0.016834,0.058053,0.104158,0.045745,0.102208,-0.007468,0.101158,0.017972,0.819495,-0.022161
2,0.0,0.000000,1.000000,-0.066532,-0.052850,-0.113437,-0.500092,-0.197902,-0.179138,-0.323675,...,0.088638,-0.252394,-0.137534,-0.091690,-0.234049,0.159373,-0.236631,0.204255,-0.500867,-0.005638
3,0.0,0.000000,0.000000,1.000000,0.231413,0.222151,0.126973,0.075906,0.038667,0.097453,...,-0.018045,0.082101,0.021076,0.011411,0.044423,0.186038,0.046296,0.225874,0.086455,0.005585
4,0.0,0.000000,0.000000,0.000000,1.000000,0.166817,-0.061912,-0.115242,0.069205,-0.077010,...,-0.004641,0.050146,0.131325,0.046695,0.040076,-0.007635,0.042812,-0.128443,-0.025749,0.060317


In [12]:
bad_cols = []

for i in range(corrs.shape[1]):
    for j in range(corrs.shape[0]):
        if (corrs.iloc[j,i] > 0.95):
            if (i != j):
                bad_cols.append(i)

In [13]:
to_drop_for_corr = list(np.unique(np.array(bad_cols)))
X.drop(X.columns[to_drop_for_corr],axis = 1, inplace = True)
test.drop(test.columns[to_drop_for_corr],axis = 1, inplace = True)

In [14]:
X.shape

(5380, 360)

In [15]:

#sc = StandardScaler()
#sc.fit(X)
#X = pd.DataFrame(sc.transform(X))
#test = pd.DataFrame(sc.transform(test))

Xknn = KNNImputer(n_neighbors=7)
Xknn.fit(X)
X = pd.DataFrame(Xknn.transform(X))
test = pd.DataFrame(Xknn.transform(test))

In [ ]:
#DO IT DIRECTLY FROM THE CORRELATION GRID
#TAKE THE UPPER TRIANGLE
#REMOVE ANYTHING WITH >0.95 CORR WITH ANYTHING


#

In [ ]:
X = add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

for i in range(len(X.columns)):
    vif_data.loc[i,'VIF'] = variance_inflation_factor(X.values, i)

print(vif_data)

In [ ]:
vif_keep = list(vif_data[(vif_data['VIF']<3) & (vif_data['feature'] != 'const')]['feature'])

X = X[vif_keep]
test = test[vif_keep]

In [ ]:
param_grid = {'max_depth': [4,6,8,10],
    'learning_rate': [0.05,0.1,0.5],
    'reg_lambda':[4,6,8],
    'n_estimators':[900,1000,1300,1500],
    'gamma':[0.1,1,10],
    'scale_pos_weight':[1,2,3],
    'subsample':[0.75,0.85,1]}

#cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)

optimal_params = RandomizedSearchCV(estimator=xgb.XGBRegressor(random_state = 1),
                                    n_iter = 25,
                                    param_distributions =param_grid,
                                    scoring = 'neg_root_mean_squared_error',
                                    verbose = 2,n_jobs=-1,cv=5)

optimal_params.fit(X,y)
optimal_params.best_score_

Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [37]:
catmodel = CatBoostRegressor().fit(X,y)
xgbmodel = xgb.XGBRegressor().fit(X,y)

preds = catmodel.predict(test)#*0.7 + xgbmodel.predict(test)*0.3

Learning rate set to 0.053413
0:	learn: 0.9828110	total: 55.2ms	remaining: 55.2s
1:	learn: 0.9673351	total: 97.5ms	remaining: 48.7s
2:	learn: 0.9544161	total: 138ms	remaining: 45.9s
3:	learn: 0.9410056	total: 178ms	remaining: 44.3s
4:	learn: 0.9290743	total: 218ms	remaining: 43.3s
5:	learn: 0.9182426	total: 258ms	remaining: 42.7s
6:	learn: 0.9081503	total: 297ms	remaining: 42.1s
7:	learn: 0.8975421	total: 337ms	remaining: 41.8s
8:	learn: 0.8877446	total: 377ms	remaining: 41.6s
9:	learn: 0.8794182	total: 417ms	remaining: 41.3s
10:	learn: 0.8715858	total: 456ms	remaining: 41s
11:	learn: 0.8638861	total: 497ms	remaining: 40.9s
12:	learn: 0.8567206	total: 536ms	remaining: 40.7s
13:	learn: 0.8494408	total: 574ms	remaining: 40.4s
14:	learn: 0.8429764	total: 609ms	remaining: 40s
15:	learn: 0.8366820	total: 645ms	remaining: 39.7s
16:	learn: 0.8308314	total: 683ms	remaining: 39.5s
17:	learn: 0.8250891	total: 721ms	remaining: 39.3s
18:	learn: 0.8193865	total: 760ms	remaining: 39.2s
19:	learn: 0.

163:	learn: 0.6335125	total: 6.72s	remaining: 34.3s
164:	learn: 0.6326432	total: 6.76s	remaining: 34.2s
165:	learn: 0.6322424	total: 6.8s	remaining: 34.2s
166:	learn: 0.6318645	total: 6.84s	remaining: 34.1s
167:	learn: 0.6314218	total: 6.88s	remaining: 34.1s
168:	learn: 0.6307692	total: 6.92s	remaining: 34s
169:	learn: 0.6302689	total: 6.95s	remaining: 33.9s
170:	learn: 0.6293978	total: 6.99s	remaining: 33.9s
171:	learn: 0.6286200	total: 7.03s	remaining: 33.8s
172:	learn: 0.6276536	total: 7.07s	remaining: 33.8s
173:	learn: 0.6266188	total: 7.11s	remaining: 33.8s
174:	learn: 0.6259085	total: 7.15s	remaining: 33.7s
175:	learn: 0.6252283	total: 7.19s	remaining: 33.7s
176:	learn: 0.6245253	total: 7.24s	remaining: 33.6s
177:	learn: 0.6238745	total: 7.28s	remaining: 33.6s
178:	learn: 0.6231758	total: 7.32s	remaining: 33.6s
179:	learn: 0.6225404	total: 7.36s	remaining: 33.5s
180:	learn: 0.6218997	total: 7.4s	remaining: 33.5s
181:	learn: 0.6209128	total: 7.44s	remaining: 33.4s
182:	learn: 0.61

326:	learn: 0.5345123	total: 13.1s	remaining: 26.9s
327:	learn: 0.5338967	total: 13.1s	remaining: 26.8s
328:	learn: 0.5332711	total: 13.1s	remaining: 26.8s
329:	learn: 0.5327951	total: 13.2s	remaining: 26.8s
330:	learn: 0.5323642	total: 13.2s	remaining: 26.7s
331:	learn: 0.5318252	total: 13.3s	remaining: 26.7s
332:	learn: 0.5313273	total: 13.3s	remaining: 26.6s
333:	learn: 0.5306706	total: 13.3s	remaining: 26.6s
334:	learn: 0.5301409	total: 13.4s	remaining: 26.5s
335:	learn: 0.5295849	total: 13.4s	remaining: 26.5s
336:	learn: 0.5288811	total: 13.4s	remaining: 26.5s
337:	learn: 0.5283682	total: 13.5s	remaining: 26.4s
338:	learn: 0.5277951	total: 13.5s	remaining: 26.4s
339:	learn: 0.5273135	total: 13.6s	remaining: 26.3s
340:	learn: 0.5267710	total: 13.6s	remaining: 26.3s
341:	learn: 0.5259722	total: 13.6s	remaining: 26.2s
342:	learn: 0.5256043	total: 13.7s	remaining: 26.2s
343:	learn: 0.5251186	total: 13.7s	remaining: 26.2s
344:	learn: 0.5245589	total: 13.8s	remaining: 26.1s
345:	learn: 

485:	learn: 0.4612959	total: 19.3s	remaining: 20.4s
486:	learn: 0.4607078	total: 19.3s	remaining: 20.3s
487:	learn: 0.4602187	total: 19.3s	remaining: 20.3s
488:	learn: 0.4598782	total: 19.4s	remaining: 20.2s
489:	learn: 0.4591514	total: 19.4s	remaining: 20.2s
490:	learn: 0.4586435	total: 19.5s	remaining: 20.2s
491:	learn: 0.4580390	total: 19.5s	remaining: 20.1s
492:	learn: 0.4576218	total: 19.5s	remaining: 20.1s
493:	learn: 0.4569451	total: 19.6s	remaining: 20s
494:	learn: 0.4563609	total: 19.6s	remaining: 20s
495:	learn: 0.4559860	total: 19.6s	remaining: 20s
496:	learn: 0.4557701	total: 19.7s	remaining: 19.9s
497:	learn: 0.4553133	total: 19.7s	remaining: 19.9s
498:	learn: 0.4550218	total: 19.8s	remaining: 19.8s
499:	learn: 0.4547076	total: 19.8s	remaining: 19.8s
500:	learn: 0.4543841	total: 19.8s	remaining: 19.8s
501:	learn: 0.4540715	total: 19.9s	remaining: 19.7s
502:	learn: 0.4535568	total: 19.9s	remaining: 19.7s
503:	learn: 0.4531059	total: 20s	remaining: 19.6s
504:	learn: 0.452528

649:	learn: 0.3993866	total: 25.8s	remaining: 13.9s
650:	learn: 0.3989636	total: 25.8s	remaining: 13.9s
651:	learn: 0.3987622	total: 25.9s	remaining: 13.8s
652:	learn: 0.3985088	total: 25.9s	remaining: 13.8s
653:	learn: 0.3979865	total: 25.9s	remaining: 13.7s
654:	learn: 0.3976434	total: 26s	remaining: 13.7s
655:	learn: 0.3974694	total: 26s	remaining: 13.7s
656:	learn: 0.3971936	total: 26.1s	remaining: 13.6s
657:	learn: 0.3966778	total: 26.1s	remaining: 13.6s
658:	learn: 0.3963100	total: 26.2s	remaining: 13.5s
659:	learn: 0.3961138	total: 26.2s	remaining: 13.5s
660:	learn: 0.3956603	total: 26.2s	remaining: 13.5s
661:	learn: 0.3956382	total: 26.3s	remaining: 13.4s
662:	learn: 0.3953022	total: 26.3s	remaining: 13.4s
663:	learn: 0.3948469	total: 26.4s	remaining: 13.3s
664:	learn: 0.3946018	total: 26.4s	remaining: 13.3s
665:	learn: 0.3941282	total: 26.4s	remaining: 13.3s
666:	learn: 0.3937966	total: 26.5s	remaining: 13.2s
667:	learn: 0.3935359	total: 26.5s	remaining: 13.2s
668:	learn: 0.39

813:	learn: 0.3474745	total: 32.3s	remaining: 7.38s
814:	learn: 0.3471843	total: 32.3s	remaining: 7.34s
815:	learn: 0.3468805	total: 32.4s	remaining: 7.3s
816:	learn: 0.3465602	total: 32.4s	remaining: 7.26s
817:	learn: 0.3462778	total: 32.5s	remaining: 7.22s
818:	learn: 0.3458729	total: 32.5s	remaining: 7.18s
819:	learn: 0.3455778	total: 32.5s	remaining: 7.14s
820:	learn: 0.3453192	total: 32.6s	remaining: 7.1s
821:	learn: 0.3449144	total: 32.6s	remaining: 7.06s
822:	learn: 0.3446480	total: 32.7s	remaining: 7.02s
823:	learn: 0.3444334	total: 32.7s	remaining: 6.98s
824:	learn: 0.3441983	total: 32.7s	remaining: 6.94s
825:	learn: 0.3438809	total: 32.8s	remaining: 6.9s
826:	learn: 0.3435759	total: 32.8s	remaining: 6.86s
827:	learn: 0.3433085	total: 32.9s	remaining: 6.82s
828:	learn: 0.3430019	total: 32.9s	remaining: 6.78s
829:	learn: 0.3425102	total: 32.9s	remaining: 6.75s
830:	learn: 0.3422735	total: 33s	remaining: 6.71s
831:	learn: 0.3418729	total: 33s	remaining: 6.67s
832:	learn: 0.34143

976:	learn: 0.3027382	total: 38.7s	remaining: 910ms
977:	learn: 0.3024422	total: 38.7s	remaining: 871ms
978:	learn: 0.3022866	total: 38.7s	remaining: 831ms
979:	learn: 0.3021125	total: 38.8s	remaining: 791ms
980:	learn: 0.3019113	total: 38.8s	remaining: 752ms
981:	learn: 0.3015381	total: 38.8s	remaining: 712ms
982:	learn: 0.3013307	total: 38.9s	remaining: 673ms
983:	learn: 0.3010654	total: 38.9s	remaining: 633ms
984:	learn: 0.3008513	total: 39s	remaining: 593ms
985:	learn: 0.3005758	total: 39s	remaining: 554ms
986:	learn: 0.3001999	total: 39s	remaining: 514ms
987:	learn: 0.2998211	total: 39.1s	remaining: 474ms
988:	learn: 0.2996109	total: 39.1s	remaining: 435ms
989:	learn: 0.2993550	total: 39.1s	remaining: 395ms
990:	learn: 0.2990165	total: 39.2s	remaining: 356ms
991:	learn: 0.2988283	total: 39.2s	remaining: 316ms
992:	learn: 0.2986141	total: 39.3s	remaining: 277ms
993:	learn: 0.2984305	total: 39.3s	remaining: 237ms
994:	learn: 0.2982806	total: 39.3s	remaining: 198ms
995:	learn: 0.2980

In [38]:
preds = np.exp(preds)

In [39]:
predictions = pd.DataFrame({'id':test_ids,'y':preds})
predictions.head()

,id,y
0,5380,4.165990
1,5381,4.295871
2,5382,3.024429
3,5383,5.368757
4,5384,2.723528


In [26]:
 # Set the parameters by cross-validation
tuned_parameters = {'n_estimators': [200, 300, 500], 'min_samples_split': [2, 4, 8]}

# clf = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=1, verbose=1)
clf = GridSearchCV(RandomForestRegressor(), tuned_parameters, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=4)
clf.fit(X, y)
print(clf.best_estimator_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 2/3] END min_samples_split=2, n_estimators=300;, score=-0.735 total time= 8.5min
[CV 2/3] END min_samples_split=4, n_estimators=200;, score=-0.735 total time= 5.4min
[CV 1/3] END min_samples_split=8, n_estimators=200;, score=-0.759 total time= 5.6min
[CV 3/3] END min_samples_split=2, n_estimators=200;, score=-0.757 total time= 5.5min
[CV 3/3] END min_samples_split=8, n_estimators=200;, score=-0.754 total time= 5.7min
[CV 1/3] END min_samples_split=2, n_estimators=200;, score=-0.759 total time= 5.5min
[CV 2/3] END min_samples_split=8, n_estimators=200;, score=-0.735 total time= 5.7min
[CV 2/3] END min_samples_split=2, n_estimators=200;, score=-0.734 total time= 5.5min
[CV 1/3] END min_samples_split=8, n_estimators=300;, score=-0.759 total time= 8.0min
[CV 1/3] END min_samples_split=4, n_estimators=500;, score=-0.757 total time=13.7min
[CV 1/3] END min_samples_split=2, n_estimators=500;, score=-0.757 total time=14.0min
[CV 3

In [25]:
clf.best_score_

0.4460983082475319

In [40]:
predictions.to_csv('catxgb1.csv',index=False) #:( 10.7